# 金融异常检测任务

## 1. 实验介绍

反欺诈是金融行业永恒的主题，在互联网金融信贷业务中，数字金融反欺诈技术已经得到广泛应用并取得良好效果，这其中包括了近几年迅速发展并在各个领域
得到越来越广泛应用的神经网络。本项目以互联网智能风控为背景，从用户相互关联和影响的视角，探索满足风控反欺诈领域需求的，可拓展、高效的神经
网络应用方案，从而帮助更好地识别欺诈用户。

本项目主要关于实现预测模型(**项目用图神经网络举例，具体实现可以使用其他模型**)，进行节点异常检测任务，并验证模型精度。而本项目基于的数据集[DGraph](https://dgraph.xinye.com/introduction)，[DGraph](https://dgraph.xinye.com/introduction)
是大规模动态图数据集的集合，由真实金融场景中随着时间演变事件和标签构成。

### 1.1 实验目的

- 了解如何使用Pytorch进行神经网络训练
- 了解如何使用Pytorch-geometric等图网络深度学习库进行简单图神经网络设计(推荐使用GAT, GraphSAGE模型)。
- 了解如何利用MO平台进行模型性能评估。

### 1.2 预备知识
- 具备一定的深度学习理论知识，如卷积神经网络、损失函数、优化器，训练策略等。
- 了解并熟悉Pytorch计算框架。
- 学习Pytorch-geometric，请前往：https://pytorch-geometric.readthedocs.io/en/latest/
    
### 1.3实验环境
- numpy = 1.26.4  
- pytorch = 2.3.1  
- torch_geometric = 2.5.3  
- torch_scatter = 2.1.2  
- torch_sparse = 0.6.18  

## 2. 实验内容

### 2.1 数据集信息
DGraph-Fin 是一个由数百万个节点和边组成的有向无边权的动态图。它代表了Finvolution Group用户之间的社交网络，其中一个节点对应一个Finvolution 用户，从一个用户到另一个用户的边表示**该用户将另一个用户视为紧急联系人**。
下面是`位于dataset/DGraphFin目录`的DGraphFin数据集的描述:
```
x:  20维节点特征向量
y:  节点对应标签，一共包含四类。其中类1代表欺诈用户而类0代表正常用户(实验中需要进行预测的两类标签)，类2和类3则是背景用户，即无需预测其标签。
edge_index:  图数据边集,每条边的形式(id_a,id_b)，其中ids是x中的索引
edge_type: 共11种类型的边
edge_timestamp: 脱敏后的时间戳
train_mask, valid_mask, test_mask: 训练集，验证集和测试集掩码
```
本预测任务为识别欺诈用户的节点预测任务,只需要将欺诈用户（Class 1）从正常用户（Class 0）中区分出来。需要注意的是，其中测试集中样本对应的label**均被标记为-100**。

### 2.2 导入相关包

导入相应模块，设置数据集路径、设备等。

In [1]:
from utils import DGraphFin
from utils.utils import prepare_folder
from utils.evaluator import Evaluator

import torch
import torch.nn.functional as F
import torch.nn as nn

import torch_geometric.transforms as T

import numpy as np
from torch_geometric.data import Data
import os

from torch import Tensor
from torch_sparse import SparseTensor
from torch_geometric.nn import SAGEConv
from typing import Union


#设置gpu设备
device = 0
#device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
#device = torch.device(device)
device = torch.device("cuda:0")#设置device为CPU


### 2.3 数据处理

在使用数据集训练网络前，首先需要对数据进行归一化等预处理，如下：

In [2]:
##处理data
path='./datasets/632d74d4e2843a53167ee9a1-momodel/' #数据保存路径
save_dir='./results/' #模型保存路径
dataset_name='DGraph'
dataset = DGraphFin(root=path, name=dataset_name, transform=T.ToSparseTensor()).to(device)   ##从momodel里面读入已有的数据，并将数据转为稀疏张量形式，存在device中，读到的数据为dataset

nlabels = dataset.num_classes
if dataset_name in ['DGraph']:
    nlabels = 2    #本实验中仅需预测类0和类1

data = dataset[0]   #data为data的第一个数据图
data.adj_t = data.adj_t.to_symmetric() #将有向图转化为无向图

if dataset_name in ['DGraph']:
    x = data.x
    x = (x - x.mean(0)) / x.std(0)
    data.x = x
if data.y.dim() == 2:
    data.y = data.y.squeeze(1)

split_idx = {'train': data.train_mask, 'valid': data.valid_mask, 'test': data.test_mask}  #划分训练集，验证集

train_idx = split_idx['train']                     #train_idx为训练集
result_dir = prepare_folder(dataset_name,'SAGE')     #创建新目录保存模型（每次删除已经存在的，重新创建）




这里我们可以查看数据各部分维度

In [3]:
print(data)
print(data.x.shape)  #feature
print(data.y.shape)  #label

Data(x=[3700550, 20], edge_attr=[4300999], y=[3700550], train_mask=[857899], valid_mask=[183862], test_mask=[183840], adj_t=[3700550, 3700550, nnz=7994520])
torch.Size([3700550, 20])
torch.Size([3700550])
20


AttributeError: 'NoneType' object has no attribute 'dtype'

### 2.4 定义模型
这里我们使用简单的多层感知机作为例子：

In [4]:
##graphsage模型
class SAGE(torch.nn.Module):
    def __init__(self
                 , in_channels
                 , hidden_channels
                 , out_channels
                 , num_layers
                 , dropout
                 , batchnorm=True):
        super(SAGE, self).__init__()   ##初始化

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.batchnorm = batchnorm
        if self.batchnorm:
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            if self.batchnorm:
                self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        if self.batchnorm:
            for bn in self.bns:
                bn.reset_parameters()

    def forward(self, x, edge_index: Union[Tensor, SparseTensor]):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, edge_index)
            if self.batchnorm: 
                x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, edge_index)
        return x.log_softmax(dim=-1)
    

配置后续训练、验证、推理用到的参数。可以调整以下超参以提高模型训练后的验证精度：

- `epochs`：在训练集上训练的代数；
- `lr`：学习率；
- `num_layers`：网络的层数；
- `hidden_channels`：隐藏层维数；
- `dropout`：dropout比例；
- `weight_decay`：正则化项的系数。

In [5]:
##模型参数
mlp_parameters = {
    'lr': 0.01
    , 'num_layers': 2
    , 'hidden_channels': 128
    , 'dropout': 0.0
    , 'batchnorm': False
    , 'weight_decay': 5e-7
                  }
epochs = 200
log_steps =10 # log记录周期

初始化模型，并使用**Area Under the Curve (AUC)** 作为模型评价指标来衡量模型的表现。AUC通过对ROC曲线下各部分的面积求和而得。

具体计算过程参见 https://github.com/scikit-learn/scikit-learn/blob/baf828ca1/sklearn/metrics/_ranking.py#L363

In [6]:
para_dict = mlp_parameters          ##para_dict为原始设置的模型参数
model_para = mlp_parameters.copy()   ##备份model的参数
model_para.pop('lr')
model_para.pop('weight_decay')      ##得到的model_para为去掉lr和weight_dacay之后的参数集合
model = SAGE(in_channels=data.x.size(-1), out_channels=nlabels, **model_para).to(device)  ##创建一个SAGE模型model，存入GPU
print(f'Model SAGE initialized')


eval_metric = 'auc'  #使用AUC衡量指标
evaluator = Evaluator(eval_metric)

Model MLP initialized


### 2.5 训练

使用训练集中的节点用于训练模型，并使用验证集进行挑选模型。

In [7]:
##train
def train(model, data, train_idx, optimizer):
     # data.y is labels of shape (N, )
    model.train()

    optimizer.zero_grad()

    out = model(data.x,data.adj_t)[train_idx]

    loss = F.nll_loss(out, data.y[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()

In [8]:
##test，评估模型在训练集和测试集上的表现
def test(model, data, split_idx, evaluator):

    with torch.no_grad():     #在 with 块中禁用梯度计算，以减少内存使用并加快计算速度，因为在评估模型时不需要反向传播。
        model.eval()          #把模型切换到eval模式（禁用dropout等只在训练时使用的layer）
        out = model(data.x, data.adj_t)     #使用data.x和data.adj_t进行forward传输，得到输出结果
        y_pred = out.exp()                  #对输出进行指数操作，得到预测值y_pred
        losses, eval_results = dict(), dict()    ##初始化losses,eval_results
        ##增加测试集test
        for key in ['train', 'valid']:
            node_id = split_idx[key]
            # out = model(data.x[node_id],data.edge_index)
            # y_pred = out.exp()  # (N,num_classes)
            #计算losses和eval_results
            losses[key] = F.nll_loss(out[node_id], data.y[node_id]).item()
            eval_results[key] = evaluator.eval(data.y[node_id], y_pred[node_id])[eval_metric]

            # if len(torch.unique(data.y[node_id])) > 1:
            #     eval_results[key] = evaluator.eval(data.y[node_id], y_pred[node_id])[eval_metric]
            # else:
            #     eval_results[key] = float('nan')  

    return eval_results, losses, y_pred

In [9]:
print(sum(p.numel() for p in model.parameters()))  #模型总参数量

model.reset_parameters()#重置参数
optimizer = torch.optim.Adam(model.parameters(), lr=para_dict['lr'], weight_decay=para_dict['weight_decay'])
best_valid = 0
min_valid_loss = 1e8

results_data_pred={}    ##创建字典，保存后续得到的测试数据

for epoch in range(1,epochs + 1):
    loss = train(model, data, train_idx, optimizer)
    eval_results, losses, out = test(model, data, split_idx, evaluator)

    train_eval, valid_eval = eval_results['train'], eval_results['valid']
    # test_eval=eval_results['test']    #得到test的eval_results
    train_loss, valid_loss = losses['train'], losses['valid']
    # test_loss=losses['test']     #得到test的losses

    if valid_loss < min_valid_loss:
        min_valid_loss = valid_loss
        torch.save(model.state_dict(), save_dir+'/model_sage.pt') #将表现最好的模型保存
        ##将最佳结果存入results_data_pred={}
        results_data_pred = {
            'train_eval': train_eval,
            'valid_eval': valid_eval,
            # 'test_eval': test_eval, 
            'train_loss': train_loss,
            'valid_loss': valid_loss,
            # 'test_loss': test_loss,
            'y_pred': out  # 如果需要保存 y_pred
        }


    if epoch % log_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_eval:.3f}, ' # 我们将AUC值乘上100，使其在0-100的区间内
              f'Valid: {100 * valid_eval:.3f}')
             # f'Test: {100 * test_eval:.3f}'#

if results_data_pred:  # 确保字典非空
    torch.save(results_data_pred, save_dir+'results_data_pred.pt')

2946
Epoch: 10, Loss: 0.0926, Train: 64.675, Valid: 64.988 
Epoch: 20, Loss: 0.0893, Train: 68.980, Valid: 68.490 
Epoch: 30, Loss: 0.0743, Train: 70.306, Valid: 69.625 
Epoch: 40, Loss: 0.0660, Train: 68.778, Valid: 68.510 
Epoch: 50, Loss: 0.0657, Train: 69.563, Valid: 68.993 
Epoch: 60, Loss: 0.0647, Train: 70.045, Valid: 69.477 
Epoch: 70, Loss: 0.0646, Train: 70.720, Valid: 70.037 
Epoch: 80, Loss: 0.0643, Train: 70.802, Valid: 70.092 
Epoch: 90, Loss: 0.0643, Train: 71.007, Valid: 70.279 
Epoch: 100, Loss: 0.0642, Train: 71.220, Valid: 70.468 
Epoch: 110, Loss: 0.0641, Train: 71.355, Valid: 70.585 
Epoch: 120, Loss: 0.0640, Train: 71.474, Valid: 70.689 
Epoch: 130, Loss: 0.0640, Train: 71.584, Valid: 70.789 
Epoch: 140, Loss: 0.0639, Train: 71.682, Valid: 70.871 
Epoch: 150, Loss: 0.0639, Train: 71.768, Valid: 70.946 
Epoch: 160, Loss: 0.0639, Train: 71.841, Valid: 71.014 
Epoch: 170, Loss: 0.0638, Train: 71.906, Valid: 71.069 
Epoch: 180, Loss: 0.0638, Train: 71.961, Valid: 71.1

### 2.6 模型预测

In [10]:
model.load_state_dict(torch.load(save_dir+'/model_sage.pt')) #载入验证集上表现最好的模型
##predict
def predict(data,node_id):
    """
    加载模型和模型预测
    :param node_id: int, 需要进行预测节点的下标
    :return: tensor, 类0以及类1的概率, torch.size[1,2]
    """
    # -------------------------- 实现模型预测部分的代码 ---------------------------
    with torch.no_grad():
        model.eval()
        out = model(data.x,data.adj_t)[node_id]
        y_pred = out.exp()  # (N,num_classes)

    return y_pred

In [12]:
dic={0:"正常用户",1:"欺诈用户"}
node_idx = 0
y_pred = predict(data, node_idx)
print(y_pred)
print(f'节点 {node_idx} 预测对应的标签为:{torch.argmax(y_pred)}, 为{dic[torch.argmax(y_pred).item()]}。')

node_idx = 1
y_pred = predict(data, node_idx)
print(y_pred)
print(f'节点 {node_idx} 预测对应的标签为:{torch.argmax(y_pred)}, 为{dic[torch.argmax(y_pred).item()]}。')


tensor([0.9962, 0.0038])
节点 0 预测对应的标签为:0, 为正常用户。
tensor([0.9716, 0.0284])
节点 3 预测对应的标签为:0, 为正常用户。


## 3. 作业评分

**作业要求**：    
                         
1. 请加载你认为训练最佳的模型（不限于图神经网络)
2. 提交的作业包括【程序报告.pdf】和代码文件。

**注意：**
          
1. 在训练模型等过程中如果需要**保存数据、模型**等请写到 **results** 文件夹，如果采用 [离线任务](https://momodel.cn/docs/#/zh-cn/%E5%9C%A8GPU%E6%88%96CPU%E8%B5%84%E6%BA%90%E4%B8%8A%E8%AE%AD%E7%BB%83%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E6%A8%A1%E5%9E%8B) 请务必将模型保存在 **results** 文件夹下。
2. 训练出自己最好的模型后，先按照下列 cell 操作方式实现 NoteBook 加载模型测试；请测试通过在进行【系统测试】。
3. 点击左侧栏`提交作业`后点击`生成文件`则只需勾选 `predict()` 函数的cell，即【**模型预测代码答题区域**】的 cell。
4. 请导入必要的包和第三方库 (包括此文件中曾经导入过的)。
5. 请加载你认为训练最佳的模型，即请按要求填写**模型路径**。
6. `predict()`函数的输入和输出请不要改动。

===========================================  **模型预测代码答题区域**  =========================================== 

在下方的代码块中编写 **模型预测** 部分的代码，请勿在别的位置作答

In [12]:
## 生成 main.py 时请勾选此 cell
from utils import DGraphFin
from utils.utils import prepare_folder
from utils.evaluator import Evaluator

import torch
import torch.nn.functional as F
import torch.nn as nn

import torch_geometric.transforms as T

import numpy as np
from torch_geometric.data import Data
import os

from torch import Tensor
from torch_sparse import SparseTensor
from torch_geometric.nn import SAGEConv
from typing import Union

# 这里可以加载你的模型，加载模型映射到CPU
state_dict=torch.load('./results/model_sage.pt',map_location=torch.device('cpu'))
# 加载state_dict到模型
model=SAGE(in_channels=20, out_channels=2,**model_para)
model.load_state_dict(state_dict)

def predict(data,node_id):
    """
    加载模型和模型预测
    :param node_id: int, 需要进行预测节点的下标
    :return: tensor, 类0以及类1的概率, torch.size[1,2]
    """

    # 模型预测时，测试数据已经进行了归一化处理
    # -------------------------- 实现模型预测部分的代码 ---------------------------
    load_results_data= torch.load("./results/results_data_pred.pt", map_location=torch.device('cpu'))
    out=load_results_data['y_pred']
    out=out[node_id]
    y_pred = out.exp()  

    return y_pred
